In [1]:
import spike.spike_analysis.spike_collection as sc
import spike.spike_analysis.spike_recording as sr
import spike.spike_analysis.firing_rate_calculations as fr
import spike.spike_analysis.normalization as norm
import spike.spike_analysis.single_cell as single_cell
import spike.spike_analysis.spike_collection as collection
import spike.spike_analysis.zscoring as zscoring
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import behavior.boris_extraction as boris
import matplotlib.pyplot as plt
import pickle
import re

In [2]:
pd.set_option('display.max_colwidth', 0)  # 0 means unlimited in newer pandas versions

# Show all rows
pd.set_option("display.max_rows", None)

# Show all columns
pd.set_option("display.max_columns", None)

# Don’t truncate column contents
pd.set_option("display.max_colwidth", None)

# Expand the display to the full width of the screen
pd.set_option("display.width", 0)


psth z-score df

In [3]:
df_path = r"C:\Users\thoma\Code\ResearchCode\diff_fam_social_memory_ephys\other_peoples_sutff\Thomas\reward_comp\outputs\zscores_global_baseline.csv"

In [14]:
spike_collection_json_path = r'C:\Users\thoma\Code\ResearchCode\diff_fam_social_memory_ephys\spike_collection.json\spike_collection.json'

In [15]:
sp = sc.SpikeCollection.load_collection(spike_collection_json_path)

In [5]:
big_df = pd.read_csv(df_path)
print(big_df.head())

                                                                     Recording  \
0  20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec   
1  20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec   
2  20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec   
3  20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec   
4  20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec   

       Event name  Unit number  Global Pre-event M  Global Pre-event SD  \
0  alone_rewarded          104            0.093333             0.675813   
1  alone_rewarded           11            0.436667             1.733876   
2  alone_rewarded          122            0.053333             0.486194   
3  alone_rewarded          125            0.708333             2.031920   
4  alone_rewarded          126            3.223333             5.619800   

   Event M  Event Z-Score      sig  Event windows  
0   

In [7]:
big_df['Event windows'].astype(int)

0        40
1        40
2        40
3        40
4        40
5        40
6        40
7        40
8        40
9        40
10       40
11       40
12       40
13       40
14       40
15       40
16       40
17       40
18       40
19       40
20       40
21       40
22       40
23       40
24       40
25       40
26       40
27       40
28       40
29       40
30       40
31       40
32       40
33       40
34       40
35       40
36       40
37       40
38       40
39       40
40       40
41       40
42       40
43       40
44        9
45        9
46        9
47        9
48        9
49        9
50        9
51        9
52        9
53        9
54        9
55        9
56        9
57        9
58        9
59        9
60        9
61        9
62        9
63        9
64        9
65        9
66        1
67        1
68        1
69        1
70        1
71        1
72        1
73        1
74        1
75        1
76        1
77        1
78        1
79        1
80        1
81        1
82        1
83  

In [10]:
big_df.shape

(6677, 9)

In [13]:
len(big_df[big_df['Event windows'] <= 3])

1569

# Data Exploration and Preparation for Clustering & Decoding Analysis

## 1. Examine Available Data Structure

Let's first understand what we have in terms of:
- Event types and trial counts
- Number of units per recording  
- Current data format vs. what we need for analysis

In [17]:
# Examine event types and trial counts in current dataframe
print("=== CURRENT DATAFRAME STRUCTURE ===")
print(f"Total rows: {len(big_df)}")
print(f"Columns: {list(big_df.columns)}")
print()

print("=== EVENT TYPES AND COUNTS ===")
event_counts = big_df['Event name'].value_counts()
print(event_counts)
print()

print("=== RECORDINGS INCLUDED ===")
recording_counts = big_df['Recording'].value_counts()
print(recording_counts)
print()

print("=== UNITS PER RECORDING ===")
units_per_recording = big_df.groupby('Recording')['Unit number'].nunique().sort_values(ascending=False)
print(units_per_recording)
print(f"\nTotal unique units: {big_df['Unit number'].nunique()}")
print(f"Total recording-unit combinations: {len(big_df.groupby(['Recording', 'Unit number']))}")

=== CURRENT DATAFRAME STRUCTURE ===
Total rows: 6677
Columns: ['Recording', 'Event name', 'Unit number', 'Global Pre-event M', 'Global Pre-event SD', 'Event M', 'Event Z-Score', 'sig', 'Event windows']

=== EVENT TYPES AND COUNTS ===
Event name
high_comp                  649
low_comp                   649
lose                       620
win                        606
high_comp_lose             605
low_comp_win               547
high_comp_win              539
low_comp_lose              500
alone_rewarded             351
low_comp_lose_baseline     244
high_comp_lose_baseline    244
low_comp_win_baseline      244
overall_pretone            244
alone_rewarded_baseline    224
high_comp_win_baseline     224
high_comp_tie              187
Name: count, dtype: int64

=== RECORDINGS INCLUDED ===
Recording
20230614_114041_standard_comp_to_training_D3_subj_1-1_t1b3L_box1_merged.rec     465
20230614_114041_standard_comp_to_training_D3_subj_1-2_t2b2L_box2_merged.rec     405
20230612_101430_standard_c

In [19]:
# Examine spike collection structure
print("=== SPIKE COLLECTION STRUCTURE ===")
print(f"Number of recordings: {len(sp.recordings)}")
print()

# Check attributes of first recording
rec = sp.recordings[0]
print(f"First recording attributes: {[attr for attr in dir(rec) if not attr.startswith('_')]}")
print()

# Try different attribute names
print("=== SAMPLE RECORDING DETAILS ===")
print(f"Recording: {rec.name}")
if hasattr(rec, 'units'):
    print(f"  - Units: {len(rec.units)}")
elif hasattr(rec, 'spike_times'):
    print(f"  - Spike times keys: {list(rec.spike_times.keys())}")
if hasattr(rec, 'event_dict'):
    print(f"  - Event types: {list(rec.event_dict.keys())}")
    for event_name, windows in rec.event_dict.items():
        print(f"    {event_name}: {len(windows)} trials")

=== SPIKE COLLECTION STRUCTURE ===
Number of recordings: 39

First recording attributes: ['all_set', 'analyze', 'event_dict', 'event_firing_rates', 'event_snippets', 'freq_dict', 'good_neurons', 'labels_dict', 'load_rec_from_h5', 'name', 'path', 'phy', 'sampling_rate', 'save_metadata_to_json', 'save_rec_to_h5', 'set_event_dict', 'set_subject', 'subject', 'timestamps', 'unit_array', 'unit_event_firing_rates', 'unit_timestamps']

=== SAMPLE RECORDING DETAILS ===
Recording: 20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec
  - Event types: ['alone_rewarded', 'alone_rewarded_baseline', 'high_comp', 'high_comp_lose', 'high_comp_lose_baseline', 'high_comp_win', 'high_comp_win_baseline', 'lose', 'low_comp', 'low_comp_lose', 'low_comp_lose_baseline', 'low_comp_win', 'low_comp_win_baseline', 'overall_pretone', 'win']
    alone_rewarded: 40 trials
    alone_rewarded_baseline: 40 trials
    high_comp: 9 trials
    high_comp_lose: 1 trials
    high_comp_lose_baseline: 40 

## 2. Create Event Mappings for Analysis

Based on the event types found, we need to create mappings for:
- Competition type: 'high', 'low', 'none' (alone_rewarded)
- Outcome valence: 'win', 'lose', 'rewarded' (for alone condition)
- Time period: 'anticipation' vs 'reaction' (we'll handle this with time bins)

In [20]:
# Create event mappings for analysis
def create_event_mappings():
    """Create mappings for competition type, outcome, and combined categories"""
    
    # Define relevant events (exclude baseline events for now)
    relevant_events = [
        'alone_rewarded', 'high_comp_win', 'high_comp_lose', 
        'low_comp_win', 'low_comp_lose', 'win', 'lose'
    ]
    
    # Competition type mapping
    comp_mapping = {}
    outcome_mapping = {}
    
    for event in relevant_events:
        if 'alone' in event:
            comp_mapping[event] = 'none'
            outcome_mapping[event] = 'rewarded'
        elif 'high_comp' in event:
            comp_mapping[event] = 'high'
            if 'win' in event:
                outcome_mapping[event] = 'win'
            elif 'lose' in event:
                outcome_mapping[event] = 'lose'
            else:
                outcome_mapping[event] = 'unknown'
        elif 'low_comp' in event:
            comp_mapping[event] = 'low'
            if 'win' in event:
                outcome_mapping[event] = 'win'
            elif 'lose' in event:
                outcome_mapping[event] = 'lose'
            else:
                outcome_mapping[event] = 'unknown'
        elif event == 'win':
            comp_mapping[event] = 'any'  # Could be high or low comp
            outcome_mapping[event] = 'win'
        elif event == 'lose':
            comp_mapping[event] = 'any'  # Could be high or low comp
            outcome_mapping[event] = 'lose'
    
    return relevant_events, comp_mapping, outcome_mapping

# Create the mappings
relevant_events, comp_mapping, outcome_mapping = create_event_mappings()

print("=== EVENT MAPPINGS FOR ANALYSIS ===")
print(f"Relevant events: {relevant_events}")
print()
print("Competition mapping:")
for event, comp in comp_mapping.items():
    print(f"  {event}: {comp}")
print()
print("Outcome mapping:")
for event, outcome in outcome_mapping.items():
    print(f"  {event}: {outcome}")

=== EVENT MAPPINGS FOR ANALYSIS ===
Relevant events: ['alone_rewarded', 'high_comp_win', 'high_comp_lose', 'low_comp_win', 'low_comp_lose', 'win', 'lose']

Competition mapping:
  alone_rewarded: none
  high_comp_win: high
  high_comp_lose: high
  low_comp_win: low
  low_comp_lose: low
  win: any
  lose: any

Outcome mapping:
  alone_rewarded: rewarded
  high_comp_win: win
  high_comp_lose: lose
  low_comp_win: win
  low_comp_lose: lose
  win: win
  lose: lose


## 3. Check Data Availability for Analysis

Let's verify we have sufficient data for meaningful clustering and decoding:
- Target: >20-30 trials per category for decoding
- Need: Per-trial firing rates (not just averaged z-scores)
- Goal: Determine if we need to extract new data or can work with existing

In [21]:
# Check current data vs. requirements for analysis
print("=== DATA AVAILABILITY ASSESSMENT ===")
print()

# Filter current dataframe to relevant events
df_filtered = big_df[big_df['Event name'].isin(relevant_events)]
print(f"Rows with relevant events: {len(df_filtered)} / {len(big_df)}")
print()

# Check trial counts per event type
print("=== TRIAL COUNTS BY EVENT TYPE ===")
event_trial_counts = df_filtered.groupby('Event name')['Event windows'].first().sort_values(ascending=False)
print(event_trial_counts)
print()

# Check units available
print("=== UNIT AVAILABILITY ===")
total_units = df_filtered['Unit number'].nunique()
units_per_event = df_filtered.groupby('Event name')['Unit number'].nunique()
print(f"Total unique units: {total_units}")
print("Units per event:")
print(units_per_event)
print()

# Assess data sufficiency
print("=== ANALYSIS READINESS ASSESSMENT ===")
min_trials = event_trial_counts.min()
max_trials = event_trial_counts.max()
print(f"Trial range: {min_trials} - {max_trials} per event")
print(f"Units available: {total_units}")

if min_trials >= 20 and total_units >= 10:
    print("✓ GOOD: Sufficient data for robust analysis")
elif min_trials >= 10 and total_units >= 5:
    print("⚠ OKAY: Marginal data - results will be preliminary but usable for abstract")
else:
    print("⚠ LIMITED: Low data - focus on proof-of-concept analysis")
    
print()
print("=== WHAT WE NEED TO EXTRACT ===")
print("Current data: Averaged z-scores per unit per event")
print("Need for clustering: Time-resolved z-scores (PSTH shapes)")
print("Need for decoding: Per-trial firing rates")

=== DATA AVAILABILITY ASSESSMENT ===

Rows with relevant events: 3768 / 6677

=== TRIAL COUNTS BY EVENT TYPE ===
Event name
alone_rewarded    40
win               13
high_comp_win      8
lose               7
low_comp_lose      6
low_comp_win       5
high_comp_lose     1
Name: Event windows, dtype: int64

=== UNIT AVAILABILITY ===
Total unique units: 251
Units per event:
Event name
alone_rewarded    186
high_comp_lose    240
high_comp_win     234
lose              248
low_comp_lose     230
low_comp_win      233
win               249
Name: Unit number, dtype: int64

=== ANALYSIS READINESS ASSESSMENT ===
Trial range: 1 - 40 per event
Units available: 251
⚠ LIMITED: Low data - focus on proof-of-concept analysis

=== WHAT WE NEED TO EXTRACT ===
Current data: Averaged z-scores per unit per event
Need for clustering: Time-resolved z-scores (PSTH shapes)
Need for decoding: Per-trial firing rates


## 4. Setup Analysis Libraries

Import the required libraries for clustering and decoding analysis.

In [22]:
# Import analysis libraries
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

print("✓ Analysis libraries imported successfully")
print("Available for:")
print("  - Clustering: KMeans, AgglomerativeClustering, PCA")
print("  - Decoding: RandomForest, LogisticRegression, SVM") 
print("  - Evaluation: Cross-validation, classification metrics")
print("  - Visualization: Seaborn, matplotlib")

✓ Analysis libraries imported successfully
Available for:
  - Clustering: KMeans, AgglomerativeClustering, PCA
  - Decoding: RandomForest, LogisticRegression, SVM
  - Evaluation: Cross-validation, classification metrics
  - Visualization: Seaborn, matplotlib


## 5. Analysis Plan and Next Steps

### Current Status Summary:
- ✅ **Data loaded**: 39 recordings, 251 units, 6677 total observations
- ✅ **Events identified**: 7 relevant event types (alone_rewarded, high/low_comp_win/lose, etc.)
- ✅ **Libraries ready**: sklearn, seaborn for clustering and decoding
- ⚠️ **Data limitation**: Low trial counts (1-40 per event type) → proof-of-concept analysis

### What We Need to Extract:

**For Clustering Analysis (PSTH Shapes):**
1. Time-resolved z-scores per unit per event (not just averaged)
2. Focus on events with sufficient trials: `alone_rewarded` (40), `win` (13), `high_comp_win` (8)
3. Create feature matrix: units × time_bins_concatenated

**For Decoding Analysis (Per-trial Classification):**
1. Per-trial firing rates per unit (currently have averaged data)
2. Trial labels for competition type and outcome
3. Feature matrix: trials × units

### Recommended Next Steps:

**Step 1: Extract Time-resolved Data**
- Modify existing PSTH code to save time_dfs (time-resolved z-scores)
- Or extract directly from spike collection using existing methods

**Step 2: Focus on Viable Comparisons**
Given limited trial counts, prioritize:
- **Clustering**: Units by response patterns across `alone_rewarded`, `win`, `lose` 
- **Decoding**: `alone_rewarded` vs `high_comp_win` (40 vs 8 trials - marginal but feasible)

**Step 3: Quick Implementation**
- Use existing z-score data as starting point
- Simple clustering (KMeans, k=3-5) on current averaged data
- Basic decoding with available units/trials

This should give you proof-of-concept results suitable for an abstract while identifying what additional data extraction is needed for robust analysis.

In [23]:
# CLARIFICATION: Trial Counts vs Event Counts (Unit Responses)
print("=== TRIALS vs EVENTS CLARIFICATION ===")
print()

# TRIAL COUNTS: How many behavioral trials occurred (from spike collection)
print("🎯 TRIAL COUNTS (behavioral trials that happened):")
for event_name, windows in rec.event_dict.items():
    if event_name in relevant_events:
        print(f"   {event_name}: {len(windows)} trials")
print()

# EVENT COUNTS: How many units responded to each event (from DataFrame)
print("📊 EVENT COUNTS (units with significant responses):")
event_unit_counts = big_df[big_df['Event name'].isin(relevant_events)].groupby('Event name').size()
for event_name, count in event_unit_counts.sort_values(ascending=False).items():
    print(f"   {event_name}: {count} units responded")
print()

print("💡 KEY DIFFERENCE:")
print("   - TRIALS = behavioral occurrences (what happened)")  
print("   - EVENTS = neural responses (how many units fired)")
print("   - For analysis: Need sufficient TRIALS per category")
print("   - Each trial can have responses from multiple units")

=== TRIALS vs EVENTS CLARIFICATION ===

🎯 TRIAL COUNTS (behavioral trials that happened):
   alone_rewarded: 40 trials
   high_comp_lose: 1 trials
   high_comp_win: 8 trials
   lose: 7 trials
   low_comp_lose: 6 trials
   low_comp_win: 5 trials
   win: 13 trials

📊 EVENT COUNTS (units with significant responses):
   lose: 620 units responded
   win: 606 units responded
   high_comp_lose: 605 units responded
   low_comp_win: 547 units responded
   high_comp_win: 539 units responded
   low_comp_lose: 500 units responded
   alone_rewarded: 351 units responded

💡 KEY DIFFERENCE:
   - TRIALS = behavioral occurrences (what happened)
   - EVENTS = neural responses (how many units fired)
   - For analysis: Need sufficient TRIALS per category
   - Each trial can have responses from multiple units
